# LCEL

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

from src import utils, conf

# Environment Variables

In [3]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Params

In [4]:
conf_settings = conf.load(file="settings.yaml")
conf_settings

LLM_WORKHORSE = "gpt-4.1-mini-2025-04-14"


A chain is an instance of the Runnable inteface and each step of a chain is also an instance of the Runnable interface

# The Runnable Interface

A chain is an instance of the Runnable Interface. In addition, every step of a chain is an instance of the Runnable Interface

An instance of a Runnable Interface can be chained with other with the overloaded operator `|`

`An interface is a design template for creating classes that share common methods. The methods defined in an interface are abstract, meaning they are only outlined and lack implementation. They act as blueprints for concrete implementations`


*In LangChain everything that can be part of a chain implements the Runnable interface.
* A Runnable is an abstraction:
>“An object that takes some input, does something with it, and produces some output.”
* This makes prompts, LLMs, retrievers, parsers, custom functions… all first-class citizens that you can compose.

Rules of a Runnable: Must implement a common set of methods (sync + async + batch + streaming):

| Method              | Purpose                          |
| ------------------- | -------------------------------- |
| `.invoke(input)`    | Run on a single input (sync).    |
| `.ainvoke(input)`   | Run on a single input (async).   |
| `.batch([inputs])`  | Run on a list of inputs (sync).  |
| `.abatch([inputs])` | Run on a list of inputs (async). |
| `.stream(input)`    | Stream output chunks (sync).     |
| `.astream(input)`   | Stream output chunks (async).    |


Kinds of Runnables related to LLM chains:
* prompts
* LLMs
* retrievers
* Output parsers



https://python.langchain.com/docs/concepts/runnables/

## RunnablePassthrough, RunnableLambda, RunnableParallel

* RunnablePassthrough: It is a placeholder
* RunnableLambda: Applies a f unction
* RunnableParallel: Runs multiple Runnables in a parallel flow

In [5]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [6]:
def input_to_upper(input: str):
    output = input.upper()
    return output

chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [7]:
chain = RunnableParallel(
    {"x": RunnablePassthrough(),  # branch 1
     "y": RunnablePassthrough()  # branch 2
     }
)
    

chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

## Input methods

A Runnable can take any input data that makes sense with the `invoke` implementation.

In general strings and dictionaries are used

Data is passed by: `input` argument or by kwargs to other Runnables

In [8]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

prompt.invoke("Devops")

ChatPromptValue(messages=[HumanMessage(content='Tell me an interesting fact about Devops', additional_kwargs={}, response_metadata={})])

In [9]:
prompt.invoke({"topic": "Devops"})

ChatPromptValue(messages=[HumanMessage(content='Tell me an interesting fact about Devops', additional_kwargs={}, response_metadata={})])

In [10]:
try:
    prompt.invoke({"input": "Devops"})
except Exception as err:
    print(err)

"Input to ChatPromptTemplate is missing variables {'topic'}.  Expected: ['topic'] Received: ['input']\nNote: if you intended {topic} to be part of the string and not a variable, please escape it with double curly braces like: '{{topic}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "


## RAG Simulation

Prompt (Runnable) → LLM (Runnable)


In [11]:
def fn_retriever(query):
    return {"a": 1, "b": 2, "c": 3}[query]


chain = RunnableParallel(
    {"context": RunnableLambda(fn_retriever),
     "query": RunnablePassthrough()
     }
)

chain.invoke("a")
    


{'context': 1, 'query': 'a'}

In [12]:
prompt = RunnablePassthrough()


retri = RunnableParallel(
    {"context": RunnableLambda(fn_retriever),
     "query": RunnablePassthrough()
     }
)

chain = prompt | retri

chain.invoke("a")

{'context': 1, 'query': 'a'}

In [13]:
# Alternative syntax

prompt = RunnablePassthrough()

retri = RunnableParallel(
    {"context": RunnableLambda(fn_retriever),
     "query": RunnablePassthrough()
     }
)

chain = (prompt |
    {"context": RunnableLambda(fn_retriever),
     "query": RunnablePassthrough()
     } 
)

chain.invoke("a")

{'context': 1, 'query': 'a'}

## Assign function

`.assign()` is a method available on any Runnable (including RunnablePassthrough).

It creates a new chain that:

* Takes the original input.
* Runs one or more sub-Runnables or functions.
* Adds their outputs as new keys in the input dictionary.
* The original input is preserved.

In short: it enriches the input with extra fields.

In [15]:
chain = RunnablePassthrough.assign(
    question_length=lambda x: len(x["question"])
)

result = chain.invoke({"question": "What is LCEL?"})
print(result)

{'question': 'What is LCEL?', 'question_length': 13}


# Implement your own Runnable Interface

In [16]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)

class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"


a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

result = chain.invoke(10)
print(result)

AddTen:  10
Multiply by 2:  20
Convert to string:  40
Result: 40
